bow: 단어의 순서 고려 x, 단어의 빈도수를 기반으로 만든 행렬  
1) 단어 추출 -> 단어에 대해 정수 인덱스 설정  
2) 단어의 등장 횟수를 저장한 벡터를 생성

In [14]:
text = "정부가. 발표하는. 물가상승률과 소비자가 느끼는 물가상승률은 다르다."

In [15]:
#퀴즈1. text에서 .(마침표)을 제거

In [17]:
import re
token=re.sub('\.','', text)

In [19]:
from konlpy.tag import Okt

In [20]:
okt=Okt()

In [22]:
token=okt.morphs(token)
token

['정부', '가', '발표', '하는', '물가상승률', '과', '소비자', '가', '느끼는', '물가상승률', '은', '다르다']

In [35]:
word2index={} #{단어: 인덱스번호,...}
bow=[] #[단어빈도수, 단어빈도수,...]

In [36]:
# 단어가 word2index에 없을 경우에만 추가
for v in token: 
    if v not in word2index.keys():
        word2index[v]=len(word2index)
        bow.insert(len(word2index)-1,1)
    else:
        idx=word2index.get(v) #get 함수:키에 해당되는 값을 리턴
        bow[idx]=bow[idx]+1
print(word2index)
bow

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

In [32]:
# test = [10,20,30,40]
# test.insert(0,1)
# test

전체문서(n개 문서) = {d1, d2, ... , dn}  
단어집합(m개 단어) = {t1, t2, ... , tm}  
xij= 문서 di에 있는 단어 tj의 출현 빈도수

In [37]:
from sklearn.feature_extraction import DictVectorizer

In [38]:
v=DictVectorizer(sparse=False) #True : 텍스트 데이터를 압축해서 bow 생성(메모리 효율)
d = [{'a':1,'b':2},{'b':3,'c':1}] #[문서1, 문서2] {단어1:빈도수, 단어2:빈도수}

In [42]:
"""     a   b   c
array([[1., 2., 0.],   문서1
       [0., 3., 1.]])  문서2
"""

'     a   b   c\narray([[1., 2., 0.],   문서1\n       [0., 3., 1.]])  문서2\n'

In [44]:
print(v.fit_transform(d)) #bow가 만들어 짐
print(v.feature_names_)

[[1. 2. 0.]
 [0. 3. 1.]]
['a', 'b', 'c']


In [46]:
#bow 에 새로운 문서 데이터를 전달하여 변환
print(v.transform({'c':5, 'd':2})) #bow에는 a,b,c단어만 존재 d는 없음


[[0. 0. 5.]]


In [47]:
#CountVectorizer
#1) 문서-> 토큰 변환 2)토큰의 빈도수 3)bow 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer

In [80]:
corpus=['This is the first document.',
    'This is the second second document.',
    'And the third one document.',
    'Is this the first document?',
    'The last document?']

In [81]:

v=CountVectorizer(min_df=2, max_df=4,) #문장에서 단어의 최소 빈도수 (최소 2번 등장하면 가져오기)
v.fit(corpus)
v.vocabulary_, v.stop_words_

({'this': 2, 'is': 1, 'first': 0},
 {'and', 'document', 'last', 'one', 'second', 'the', 'third'})

In [76]:
#많이 쓰이는 기능 (아래부터 읽기)
v=CountVectorizer(ngram_range=(1,2),token_pattern="t\w+")
v.fit(corpus)
v.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

In [72]:
# ngram_range=(n,m) 의미? 최소 n개 최대 m개의 단어들을 묶어서 1개의 단어로 취급하여 bow를 생성
v=CountVectorizer(ngram_range=(2,2)) #최소길이 2, 최대길이 2 = 2(bi) gram
v=CountVectorizer(ngram_range=(1,4))
v.fit(corpus)
v.vocabulary_
# 2단어가 1개의 단어로 사용 : 'this is' = 하나의 단어로 보기.
# 점 마침표 물음표가 제외된것을 확인

{'this': 35,
 'is': 7,
 'the': 23,
 'first': 5,
 'document': 4,
 'this is': 36,
 'is the': 8,
 'the first': 24,
 'first document': 6,
 'this is the': 37,
 'is the first': 9,
 'the first document': 25,
 'this is the first': 38,
 'is the first document': 10,
 'second': 19,
 'the second': 28,
 'second second': 21,
 'second document': 20,
 'is the second': 11,
 'the second second': 29,
 'second second document': 22,
 'this is the second': 39,
 'is the second second': 12,
 'the second second document': 30,
 'and': 0,
 'third': 33,
 'one': 18,
 'and the': 1,
 'the third': 31,
 'third one': 34,
 'and the third': 2,
 'the third one': 32,
 'and the third one': 3,
 'is this': 13,
 'this the': 40,
 'is this the': 14,
 'this the first': 41,
 'is this the first': 15,
 'this the first document': 42,
 'last': 16,
 'the last': 26,
 'last document': 17,
 'the last document': 27}

In [67]:
# Option: 테크닉!!
v=CountVectorizer(token_pattern="t\w+") #t로 시작하는 문자
v.fit(corpus)
v.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

In [62]:
# tip
v=CountVectorizer(analyzer='char')
v.fit(corpus)
v.vocabulary_

{'t': 16,
 'h': 8,
 'i': 9,
 's': 15,
 ' ': 0,
 'e': 6,
 'f': 7,
 'r': 14,
 'd': 5,
 'o': 13,
 'c': 4,
 'u': 17,
 'm': 11,
 'n': 12,
 '.': 1,
 'a': 3,
 '?': 2,
 'l': 10}

In [60]:
# Option: stop_words
v=CountVectorizer(stop_words='english')
v.fit(corpus)
v.vocabulary_

{'document': 0, 'second': 1}

In [56]:
# Option: stop_words
v=CountVectorizer(stop_words=['and','is','the','this'])
v.fit(corpus)
v.vocabulary_

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}

In [61]:
# 여기서부터 위로 읽기 ^
v=CountVectorizer()
v.fit(corpus)
v.vocabulary_

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [57]:
v.transform(corpus).toarray()
#2차원 배열, 첫번째행: This is the first document -> bow
# [-,document,first,is,-,-,-,the,-,this]
# [0, 1, 1, 1, 0, 0, 0, 1, 0, 1]

array([[1, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 2, 0],
       [0, 0, 0, 1, 0, 1],
       [1, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0]])

In [58]:
# 새로운 문서가 입력 -> bow 변환
v.transform(["This is the second document"]).toarray()

array([[1, 0, 0, 0, 1, 0]])

In [59]:
v.transform(["Something new"]).toarray()

array([[0, 0, 0, 0, 0, 0]])

# tf-idf
- term frequency (tf)
- inverse document frequency (idf)
- 정보 검색, 텍스트 마이닝 및 사용자 모델링 검색에서 가중치 요소로 자주 사용됩니다. tf-idf 값은 단어가 문서에 나타나는 횟수에 비례하여 증가하고 단어를 포함하는 말뭉치의 문서 수로 오프셋됩니다. 이는 일부 단어가 일반적으로 더 자주 나타나는 사실을 조정하는 데 도움이됩니다. tf–idf는 오늘날 가장 널리 사용되는 용어 가중치 체계 중 하나입니다.

https://en.wikipedia.org/wiki/Tf–idf

- dtm에서는 각각의 단어에 대한 중요도가 없다.
    - 단어 빈도수만 저장. 중요도에대한 정보는 전혀 없다.
```
    t1... tm
d1  3
d2
.
.
.
dn

```
- 단어가 얼마나 중요한지 알고 싶다.
- tf-idf (단어빈도-역문서빈도) : dtm에 있는 각 단어들마다 중요도를 계산
- dtm을 생성한 후, tf-idf를 생성

- 주의:
    - tf(d,t) : 문서 d에서 단어 t의 등장 횟수
    - df(t) : 단어 t가 등장한 문서의 개수(여러 번 단어가 등장한 경우에도 1로 카운트)
    
    - idf(d,t): df(t)의 반비례하는 수
    
n: 전체 문서의 개수  
idf(d,t) = log(n / (1+df(t))  
  
n / (1+df(t))  
  
log를 취하는 이유:  
ex) n: 100만  
idf(d, t) = 100만 / 1 = 100만  
idf(d, t) = 100만 / 100 = 1만  
idf(d, t) = 100만 / 100만 = 1  
  
ex) n: 100만  
idf(d, t) = log(100만 / 1) = 6  
idf(d, t) = log(100만 / 100) = 4  
idf(d, t) = log(100만 / 10만) = 1  
idf(d, t) = log(100만 / 100만) = 0  

불용어가 더 많은 경우가 많다. 그러므로 로그를 씌어주지 않으면 가중치가 더 많이나와서 log를 사용하는것.  

tfidf는 모든 문서에서 자주 등장하는 단어라면 중요도가 낮다고 판단,  
특정 문서에서만 등장하는 단어라면 중요도가 높다고 판단  
tfidf값이 낮(높)으면 -> 단어의 중요도가 낮(높)다.

In [110]:
import pandas as pd
from math import log
import numpy as np

In [86]:
log(4/(2+1))

0.28768207245178085

In [87]:
#연습문제
docs = [ # 문서 4개
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]
#tf(dtm), idf, tfidf'

In [89]:
v=CountVectorizer()
v.fit(docs)
v.vocabulary_

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [164]:
dtm=v.transform(docs).toarray()
dtm
df=pd.DataFrame(dtm,columns=sorted(v.vocabulary_))
df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [165]:
temp_idf = []
for n in range(dtm.shape[1]):
    dft=np.count_nonzero(dtm[:,n])
    idf=log(dtm.shape[0]/(1+dft))
    temp_idf.append(idf)
print(temp_idf)

[0.6931471805599453, 0.6931471805599453, 0.6931471805599453, 0.28768207245178085, 0.28768207245178085, 0.6931471805599453, 0.28768207245178085, 0.6931471805599453, 0.6931471805599453]


In [166]:
pd.DataFrame(temp_idf*df)

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


In [163]:
#노용철님 (정답)
docs =[

'먹고 싶은 사과','먹고 싶은 바나나','길고 노란 바나나 바나나','저는 과일이 좋아요'

]

cv = CountVectorizer()

cv.fit(docs)

df = pd.DataFrame(cv.transform(docs).toarray(),columns =sorted(cv.vocabulary_))

df*np.log(len(df) / (1+df.apply(lambda x: x>0).sum()))

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147
